### Task

#### Q.1. When store is closed, sales = 0. Can this insight be used for Data Cleaning? Perform this and retrain the Linear Regression model. Compare the result with previous model.

In [1]:
import pandas as pd

In [2]:
training_df = pd.read_csv("..\\train_data.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
training_df.shape

(982644, 9)

In [4]:
from sklearn import preprocessing
from copy import deepcopy

training_df_encode = deepcopy(training_df)
training_df_encode['StateHoliday'] = training_df_encode['StateHoliday'].astype(str)
le = preprocessing.LabelEncoder()
training_df_encode['StateHoliday'] = le.fit_transform(training_df_encode['StateHoliday'].fillna(""))
training_df_encode['StateHoliday'] = training_df_encode['StateHoliday'].astype(int)

StateHoliday_LE_Map = dict(zip(le.classes_, le.transform(le.classes_)))
StateHoliday_LE_Map_reverse = dict(zip(le.transform(le.classes_), le.classes_))

In [5]:
test_df = pd.read_csv("..\\test_data_hidden.csv")

In [6]:
test_df['StateHoliday'] = test_df['StateHoliday'].map(StateHoliday_LE_Map_reverse)

In [7]:
test_df.shape

(34565, 9)

In [8]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def model_training(X_train, y_train):
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    return regr

In [9]:
def model_predict(X_test, regr):
    y_pred = regr.predict(X_test)
    return list(y_pred)

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def model_performance(y_test, y_pred, regr):
        
#     try:
    # The mean squared error
    mse = mean_squared_error(y_test, y_pred)

    # The mean absolute error
    mae = mean_absolute_error(y_test, y_pred)

    # The coefficient of determination: 1 is perfect prediction

    r2 = r2_score(y_test, y_pred)
        
    
    
    return mse, mae, r2

In [11]:
def model_for_all_or_each_store(training_df, test_df, store_no):
    
    if str(store_no).lower() == 'all':
        print ("training_df_all shape : ", training_df.shape)
        # Splitting Training Data
        X_train = training_df[['Store', 'Customers','Open', 'Promo', 'StateHoliday', 'SchoolHoliday']]
        y_train = training_df['Sales']

        # Splitting Testing Data
        X_test = test_df[['Store', 'Customers','Open', 'Promo', 'StateHoliday', 'SchoolHoliday']]
        y_test = list(test_df['Sales'])
        
         # Model Training
        regr = model_training(X_train, y_train)

        # Prediction
        y_pred = model_predict(X_test, regr)

        # Performance Measurement
        mse, mae, r2 = model_performance(y_test, y_pred, regr)
        
        return y_pred, mse, mae, r2
        
    else:
        training_df_store = training_df[training_df['Store'] == store_no]
        print ("training_df_store shape : ", training_df_store.shape)
        
        
        test_df = test_df[test_df['Store'] == store_no]
        
        # Splitting Training Data
        X_train = training_df_store[['Customers','Open', 'Promo', 'StateHoliday', 'SchoolHoliday']]
        y_train = training_df_store['Sales']

        # Splitting Testing Data
        X_test = test_df[['Customers','Open', 'Promo', 'StateHoliday', 'SchoolHoliday']]
        y_test = list(test_df['Sales'])
        
        # Model Training
        regr = model_training(X_train, y_train)

        # Prediction
        y_pred = model_predict(X_test, regr)

        # Performance Measurement
        mse, mae, r2 = model_performance(y_test, y_pred, regr)
        
        return y_pred, mse, mae, r2
   
    
    
    

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

training_df_encode_without_store_0 = training_df_encode[training_df_encode['Sales'] != 0]
test_df_without_store_0 = test_df[test_df['Sales'] != 0]


y_pred, mse, mae, r2 = model_for_all_or_each_store(training_df_encode_without_store_0, test_df_without_store_0, 1)
print('Mean squared error: %.2f'%mse )
print('Mean Absolute Error: %.2f'%mae)
print('R-Square Score: %.2f'%r2)

training_df_store shape :  (754, 9)
Mean squared error: 62788.86
Mean Absolute Error: 207.92
R-Square Score: 0.86


#### 2. Use Non-Linear Regressors (DT-Regressors). ####
       a)    Train a single model for all stores, where storeId can be a feature.
       b)    Train separate models for each store.


In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def model_training(X_train, y_train):
    regr = DecisionTreeRegressor(max_depth=25, random_state=42)
    regr.fit(X_train, y_train)
    return regr

In [19]:
def model_predict(X_test, regr):
    y_pred = regr.predict(X_test)
    return list(y_pred)

In [20]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred, mse, mae, r2 = model_for_all_or_each_store(training_df_encode, test_df, 1)
print('Mean squared error: %.2f'%mse )
print('Mean Absolute Error: %.2f'%mae)
print('R-Square Score: %.2f'%r2)

training_df_store shape :  (911, 9)
Mean squared error: 155805.89
Mean Absolute Error: 287.99
R-Square Score: 0.94
